In [1]:
# Initial imports
import os
import json
import requests
import pandas as pd
from pathlib import Path
import quandl
from dotenv import load_dotenv
from datetime import timedelta as td
import dateutil.parser

%matplotlib inline

In [2]:
load_dotenv()

True

In [3]:
# Set API key from env variables
quandl.ApiConfig.api_key = os.getenv("QUANDL_API")

## Indicators
The indicators table explains how the housing data has been categorized

In [4]:
# Download housing data from quadl api
indicator_df = quandl.get_table("ZILLOW/INDICATORS")

In [5]:
# Show indicators dataframe
indicator_df.head(20)

,indicator_id,indicator,category
None,,,
0,ZSFH,ZHVI Single-Family Homes Time Series ($),Home values
1,ZCON,ZHVI Condo/Co-op Time Series ($),Home values
2,ZATT,ZHVI All Homes- Top Tier Time Series ($),Home values
3,ZALL,"ZHVI All Homes (SFR, Condo/Co-op) Time Series ($)",Home values
4,ZABT,ZHVI All Homes- Bottom Tier Time Series ($),Home values
5,Z5BR,ZHVI 5+ Bedroom Time Series ($),Home values
6,Z4BR,ZHVI 4-Bedroom Time Series ($),Home values
7,Z3BR,ZHVI 3-Bedroom Time Series ($),Home values
8,Z2BR,ZHVI 2-Bedroom Time Series ($),Home values


## Regions
The regions table explains the different region types for the housing data

In [6]:
# Show regions dataframe
regions = quandl.get_table("ZILLOW/REGIONS", paginate=True)
regions.head()

,region_id,region_type,region
None,,,
0,99999,zip,98847; WA; Wenatchee; Chelan County; Peshastin
1,99998,zip,98846; WA; Okanogan County; Pateros
2,99997,zip,98845; WA; Wenatchee; Douglas County; Palisades
3,99996,zip,98844; WA; Okanogan County; Oroville
4,99995,zip,98843; WA; Wenatchee; Douglas County; Orondo


## Data Table
The data table contains all the housing data by indicator and region.

The downloaded zip is extracted to a location outside of the repository due to it's large size.

In [7]:
# -- Code commented out as it only needs to be run a single time

# -------
# data = quandl.export_table('ZILLOW/DATA', filename="../../Repositories/local/zdata.zip")
# with zipfile.ZipFile('../../Repositories/local/zdata.zip', 'r') as zip_ref:
#    zip_ref.extractall('../../Repositories/local/')
# os.rename('../../Repositories/local/ZILLOW_DATA_962c837a6ccefddddf190101e0bafdaf.csv', '../../Repositories/local/zdata.csv')
# Read extracted zip file into a dataframe
zdata = pd.read_csv('../../Repositories/local/zdata.csv')
# -------

In [8]:
zdata.shape

(127237469, 4)

In [9]:
regions.region_type.value_counts()

zip       31172
city      27471
neigh     16711
county     2887
metro       915
state        51
Name: region_type, dtype: int64

## Extract metro codes to list

In [10]:
# Casting the list values as int for use to make splits from the region_id column
metros = regions[regions.region_type == 'metro']
majors_list = ['New York, NY', 'Los Angeles-Long Beach-Anaheim, CA', 'Chicago, IL', 'Houston, TX', 'Phoenix, AZ',
               'Philadelphia, PA', 'San Antonio, TX', 'San Diego, CA', 'Dallas-Fort Worth, TX', 'San Jose, CA',
              'Austin, TX', 'Jacksonville, FL', 'Columbus, OH', 'Charlotte, NC',
              'Indianapolis, IN', 'San Francisco, CA', 'Seattle, WA', 'Denver, CO', 'Washington, DC',
              'Boston, MA', 'El Paso, TX', 'Nashville, TN', 'Detroit, MI', 'Las Vegas, NV',
               'Oklahoma City, OK', 'Portland, OR', 'Memphis, TN', 'Louisville-Jefferson County, KY', 'Milwaukee, WI',
              'Baltimore, MD']
major_metros = metros[metros.region.isin(majors_list)]
major_metros.sort_values('region')
major_metros.drop(columns='region_type', inplace=True)
major_metros = major_metros.reset_index(drop=True)
major_metros.region_id = major_metros.region_id.astype('int')
major_metros = major_metros.sort_values('region')
major_metros

C:\ProgramData\Anaconda3\envs\deep\lib\site-packages\pandas\core\frame.py:4908: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


,region_id,region
29,394355,"Austin, TX"
28,394358,"Baltimore, MD"
27,394404,"Boston, MA"
26,394458,"Charlotte, NC"
25,394463,"Chicago, IL"
24,394492,"Columbus, OH"
23,394514,"Dallas-Fort Worth, TX"
22,394530,"Denver, CO"
21,394532,"Detroit, MI"
20,394561,"El Paso, TX"


In [11]:
#finder = metros.loc[metros['region'].str.contains("Baltimore", case=False)]
#finder

## Extract home value codes to list

In [12]:
home_value_codes = []
for index, row in indicator_df.iterrows():
    if row[2] == "Home values":
        home_value_codes.append(row[0])
home_value_codes

['ZSFH',
 'ZCON',
 'ZATT',
 'ZALL',
 'ZABT',
 'Z5BR',
 'Z4BR',
 'Z3BR',
 'Z2BR',
 'Z1BR']

In [13]:
del home_value_codes[0]
del home_value_codes[1:4]
home_value_codes

['ZCON', 'Z5BR', 'Z4BR', 'Z3BR', 'Z2BR', 'Z1BR']

## Home values used to create a slice of zdata

In [14]:
# Dataframe containing all homes from 1BR to 5BR and Condos
homes = zdata[zdata.indicator_id.isin(home_value_codes)]
homes

,indicator_id,region_id,date,value
11667,ZCON,9,2020-10-31,537729.0
11960,ZCON,54,2020-10-31,185435.0
12253,ZCON,43,2020-10-31,580421.0
12546,ZCON,14,2020-10-31,196766.0
12839,ZCON,21,2020-10-31,202002.0
...,...,...,...,...
127237464,Z4BR,49589,2020-02-29,103928.0
127237465,Z4BR,49589,2020-03-31,105018.0
127237466,Z4BR,49589,2020-04-30,105616.0
127237467,Z4BR,49589,2020-06-30,105040.0


## Metro codes used to slice homes data

In [15]:
metro_homes = homes.merge(major_metros, on="region_id", how = 'inner')
metro_homes

,indicator_id,region_id,date,value,region
0,ZCON,394913,2020-06-30,498772.0,"New York, NY"
1,Z5BR,394913,2020-06-30,795913.0,"New York, NY"
2,Z2BR,394913,2020-06-30,353412.0,"New York, NY"
3,Z3BR,394913,2020-06-30,432483.0,"New York, NY"
4,Z1BR,394913,2020-06-30,406444.0,"New York, NY"
...,...,...,...,...,...
54640,Z4BR,394561,2019-10-31,188331.0,"El Paso, TX"
54641,Z4BR,394561,2019-11-30,188413.0,"El Paso, TX"
54642,Z4BR,394561,2019-12-31,189187.0,"El Paso, TX"
54643,Z4BR,394561,2020-01-31,189962.0,"El Paso, TX"


### Create Condos-Only DataFrame

In [33]:
metro_condo = metro_homes[metro_homes.indicator_id == 'ZCON']
metro_condo = metro_condo.sort_values(by=['region_id','date'])
metro_condo = metro_condo.drop(columns='indicator_id')
metro_condo = metro_condo.reindex(columns=['date', 'region_id', 'region', 'value'])
metro_condo.reset_index(drop=True, inplace=True)
metro_condo

,date,region_id,region,value
0,1996-01-31,394355,"Austin, TX",153184.0
1,1996-02-29,394355,"Austin, TX",152411.0
2,1996-03-31,394355,"Austin, TX",151035.0
3,1996-04-30,394355,"Austin, TX",149075.0
4,1996-05-31,394355,"Austin, TX",147382.0
...,...,...,...,...
9103,2021-01-31,753899,"Los Angeles-Long Beach-Anaheim, CA",584577.0
9104,2021-02-28,753899,"Los Angeles-Long Beach-Anaheim, CA",589481.0
9105,2021-03-31,753899,"Los Angeles-Long Beach-Anaheim, CA",584773.0
9106,2021-04-30,753899,"Los Angeles-Long Beach-Anaheim, CA",592553.0


In [52]:
df_names = []
for i in range(len(majors_list)):
    s = majors_list[i]
    s = s.replace(',', '')
    s = s.replace(' ', '')
    df_names.append(s)
s = df_names[1]
s = s.replace('-LongBeach-Anaheim', '')
df_names[1] = s
s = df_names[8]
s = s.replace('-FortWorth', '')
df_names[8] = s
s = df_names[27]
s = s.replace('-JeffersonCounty', '')
df_names[27] = s
df_names

['NewYorkNY',
 'LosAngelesCA',
 'ChicagoIL',
 'HoustonTX',
 'PhoenixAZ',
 'PhiladelphiaPA',
 'SanAntonioTX',
 'SanDiegoCA',
 'DallasTX',
 'SanJoseCA',
 'AustinTX',
 'JacksonvilleFL',
 'ColumbusOH',
 'CharlotteNC',
 'IndianapolisIN',
 'SanFranciscoCA',
 'SeattleWA',
 'DenverCO',
 'WashingtonDC',
 'BostonMA',
 'ElPasoTX',
 'NashvilleTN',
 'DetroitMI',
 'LasVegasNV',
 'OklahomaCityOK',
 'PortlandOR',
 'MemphisTN',
 'LouisvilleKY',
 'MilwaukeeWI',
 'BaltimoreMD']

In [66]:
NewYorkNY_condo_df = pd.DataFrame()
temp_df = metro_condo[metro_condo.region == majors_list[0]]
temp_df['change'] = temp_df['value'].pct_change() * 100
temp_df['lagged_change'] = temp_df['change'].shift()
NewYorkNY_condo_df = pd.concat([NewYorkNY_condo_df, temp_df])

NewYorkNY_condo_df

C:\ProgramData\Anaconda3\envs\deep\lib\site-packages\pandas\core\frame.py:3607: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._set_item(key, value)


,date,region_id,region,value,change,lagged_change
5448,1996-01-31,394913,"New York, NY",165513.0,NaN,NaN
5449,1996-02-29,394913,"New York, NY",165142.0,-0.224152,NaN
5450,1996-03-31,394913,"New York, NY",164931.0,-0.127769,-0.224152
5451,1996-04-30,394913,"New York, NY",164561.0,-0.224336,-0.127769
5452,1996-05-31,394913,"New York, NY",164388.0,-0.105128,-0.224336
...,...,...,...,...,...,...
5748,2021-01-31,394913,"New York, NY",492765.0,-1.095096,-0.348625
5749,2021-02-28,394913,"New York, NY",492527.0,-0.048299,-1.095096
5750,2021-03-31,394913,"New York, NY",494653.0,0.431651,-0.048299
5751,2021-04-30,394913,"New York, NY",496172.0,0.307084,0.431651


In [67]:
LosAngelesCA_condo_df = pd.DataFrame()
temp_df = metro_condo[metro_condo.region == majors_list[1]]
temp_df['change'] = temp_df['value'].pct_change() * 100
temp_df['lagged_change'] = temp_df['change'].shift()
LosAngelesCA_condo_df = pd.concat([LosAngelesCA_condo_df, temp_df])

LosAngelesCA_condo_df

C:\ProgramData\Anaconda3\envs\deep\lib\site-packages\pandas\core\frame.py:3607: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._set_item(key, value)


,date,region_id,region,value,change,lagged_change
8803,1996-01-31,753899,"Los Angeles-Long Beach-Anaheim, CA",153716.0,NaN,NaN
8804,1996-02-29,753899,"Los Angeles-Long Beach-Anaheim, CA",153804.0,0.057248,NaN
8805,1996-03-31,753899,"Los Angeles-Long Beach-Anaheim, CA",153657.0,-0.095576,0.057248
8806,1996-04-30,753899,"Los Angeles-Long Beach-Anaheim, CA",153447.0,-0.136668,-0.095576
8807,1996-05-31,753899,"Los Angeles-Long Beach-Anaheim, CA",153124.0,-0.210496,-0.136668
...,...,...,...,...,...,...
9103,2021-01-31,753899,"Los Angeles-Long Beach-Anaheim, CA",584577.0,0.896819,1.434912
9104,2021-02-28,753899,"Los Angeles-Long Beach-Anaheim, CA",589481.0,0.838897,0.896819
9105,2021-03-31,753899,"Los Angeles-Long Beach-Anaheim, CA",584773.0,-0.798669,0.838897
9106,2021-04-30,753899,"Los Angeles-Long Beach-Anaheim, CA",592553.0,1.330431,-0.798669


In [69]:
ChicagoIL_condo_df = pd.DataFrame()
temp_df = metro_condo[metro_condo.region == majors_list[2]]
temp_df['change'] = temp_df['value'].pct_change() * 100
temp_df['lagged_change'] = temp_df['change'].shift()
ChicagoIL_condo_df = pd.concat([ChicagoIL_condo_df, temp_df])

ChicagoIL_condo_df

C:\ProgramData\Anaconda3\envs\deep\lib\site-packages\pandas\core\frame.py:3607: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._set_item(key, value)


,date,region_id,region,value,change,lagged_change
1220,1996-01-31,394463,"Chicago, IL",135785.0,NaN,NaN
1221,1996-02-29,394463,"Chicago, IL",135573.0,-0.156129,NaN
1222,1996-03-31,394463,"Chicago, IL",135330.0,-0.179239,-0.156129
1223,1996-04-30,394463,"Chicago, IL",135097.0,-0.172172,-0.179239
1224,1996-05-31,394463,"Chicago, IL",134739.0,-0.264995,-0.172172
...,...,...,...,...,...,...
1520,2021-01-31,394463,"Chicago, IL",216886.0,1.203414,2.254975
1521,2021-02-28,394463,"Chicago, IL",219591.0,1.247199,1.203414
1522,2021-03-31,394463,"Chicago, IL",220501.0,0.414407,1.247199
1523,2021-04-30,394463,"Chicago, IL",222264.0,0.799543,0.414407


In [72]:
HoustonTX_condo_df = pd.DataFrame()
temp_df = metro_condo[metro_condo.region == majors_list[3]]
temp_df['change'] = temp_df['value'].pct_change() * 100
temp_df['lagged_change'] = temp_df['change'].shift()
HoustonTX_condo_df = pd.concat([HoustonTX_condo_df, temp_df])

HoustonTX_condo_df

C:\ProgramData\Anaconda3\envs\deep\lib\site-packages\pandas\core\frame.py:3607: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._set_item(key, value)


,date,region_id,region,value,change,lagged_change
3050,1996-01-31,394692,"Houston, TX",82165.0,NaN,NaN
3051,1996-02-29,394692,"Houston, TX",82546.0,0.463701,NaN
3052,1996-03-31,394692,"Houston, TX",82595.0,0.059361,0.463701
3053,1996-04-30,394692,"Houston, TX",82936.0,0.412858,0.059361
3054,1996-05-31,394692,"Houston, TX",83071.0,0.162776,0.412858
...,...,...,...,...,...,...
3350,2021-01-31,394692,"Houston, TX",146789.0,-0.382078,1.495375
3351,2021-02-28,394692,"Houston, TX",149422.0,1.793731,-0.382078
3352,2021-03-31,394692,"Houston, TX",152348.0,1.958212,1.793731
3353,2021-04-30,394692,"Houston, TX",153792.0,0.947830,1.958212


In [74]:
PhoenixAZ_condo_df = pd.DataFrame()
temp_df = metro_condo[metro_condo.region == majors_list[4]]
temp_df['change'] = temp_df['value'].pct_change() * 100
temp_df['lagged_change'] = temp_df['change'].shift()
PhoenixAZ_condo_df = pd.concat([PhoenixAZ_condo_df, temp_df])

PhoenixAZ_condo_df

C:\ProgramData\Anaconda3\envs\deep\lib\site-packages\pandas\core\frame.py:3607: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._set_item(key, value)


,date,region_id,region,value,change,lagged_change
6363,1996-01-31,394976,"Phoenix, AZ",78671.0,NaN,NaN
6364,1996-02-29,394976,"Phoenix, AZ",78864.0,0.245325,NaN
6365,1996-03-31,394976,"Phoenix, AZ",79077.0,0.270085,0.245325
6366,1996-04-30,394976,"Phoenix, AZ",79510.0,0.547568,0.270085
6367,1996-05-31,394976,"Phoenix, AZ",79950.0,0.553390,0.547568
...,...,...,...,...,...,...
6663,2021-01-31,394976,"Phoenix, AZ",223699.0,3.125115,1.351699
6664,2021-02-28,394976,"Phoenix, AZ",228447.0,2.122495,3.125115
6665,2021-03-31,394976,"Phoenix, AZ",232782.0,1.897596,2.122495
6666,2021-04-30,394976,"Phoenix, AZ",237883.0,2.191321,1.897596


In [75]:
PhiladelphiaPA_condo_df = pd.DataFrame()
temp_df = metro_condo[metro_condo.region == majors_list[5]]
temp_df['change'] = temp_df['value'].pct_change() * 100
temp_df['lagged_change'] = temp_df['change'].shift()
PhiladelphiaPA_condo_df = pd.concat([PhiladelphiaPA_condo_df, temp_df])

PhiladelphiaPA_condo_df

C:\ProgramData\Anaconda3\envs\deep\lib\site-packages\pandas\core\frame.py:3607: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._set_item(key, value)


,date,region_id,region,value,change,lagged_change
6058,1996-01-31,394974,"Philadelphia, PA",95171.0,NaN,NaN
6059,1996-02-29,394974,"Philadelphia, PA",94985.0,-0.195438,NaN
6060,1996-03-31,394974,"Philadelphia, PA",94811.0,-0.183187,-0.195438
6061,1996-04-30,394974,"Philadelphia, PA",94513.0,-0.314310,-0.183187
6062,1996-05-31,394974,"Philadelphia, PA",94299.0,-0.226424,-0.314310
...,...,...,...,...,...,...
6358,2021-01-31,394974,"Philadelphia, PA",218120.0,0.704079,0.567855
6359,2021-02-28,394974,"Philadelphia, PA",220254.0,0.978361,0.704079
6360,2021-03-31,394974,"Philadelphia, PA",222375.0,0.962979,0.978361
6361,2021-04-30,394974,"Philadelphia, PA",225249.0,1.292411,0.962979


In [76]:
SanAntonioTX_condo_df = pd.DataFrame()
temp_df = metro_condo[metro_condo.region == majors_list[6]]
temp_df['change'] = temp_df['value'].pct_change() * 100
temp_df['lagged_change'] = temp_df['change'].shift()
SanAntonioTX_condo_df = pd.concat([SanAntonioTX_condo_df, temp_df])

SanAntonioTX_condo_df

C:\ProgramData\Anaconda3\envs\deep\lib\site-packages\pandas\core\frame.py:3607: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._set_item(key, value)


,date,region_id,region,value,change,lagged_change
6973,1996-01-31,395055,"San Antonio, TX",82329.0,NaN,NaN
6974,1996-02-29,395055,"San Antonio, TX",82284.0,-0.054659,NaN
6975,1996-03-31,395055,"San Antonio, TX",82233.0,-0.061980,-0.054659
6976,1996-04-30,395055,"San Antonio, TX",82181.0,-0.063235,-0.061980
6977,1996-05-31,395055,"San Antonio, TX",82103.0,-0.094912,-0.063235
...,...,...,...,...,...,...
7273,2021-01-31,395055,"San Antonio, TX",161595.0,2.035082,0.703267
7274,2021-02-28,395055,"San Antonio, TX",157801.0,-2.347845,2.035082
7275,2021-03-31,395055,"San Antonio, TX",160059.0,1.430916,-2.347845
7276,2021-04-30,395055,"San Antonio, TX",162404.0,1.465085,1.430916


In [77]:
SanDiegoCA_condo_df = pd.DataFrame()
temp_df = metro_condo[metro_condo.region == majors_list[7]]
temp_df['change'] = temp_df['value'].pct_change() * 100
temp_df['lagged_change'] = temp_df['change'].shift()
SanDiegoCA_condo_df = pd.concat([SanDiegoCA_condo_df, temp_df])

SanDiegoCA_condo_df

C:\ProgramData\Anaconda3\envs\deep\lib\site-packages\pandas\core\frame.py:3607: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._set_item(key, value)


,date,region_id,region,value,change,lagged_change
7278,1996-01-31,395056,"San Diego, CA",133652.0,NaN,NaN
7279,1996-02-29,395056,"San Diego, CA",133301.0,-0.262622,NaN
7280,1996-03-31,395056,"San Diego, CA",133061.0,-0.180044,-0.262622
7281,1996-04-30,395056,"San Diego, CA",132569.0,-0.369755,-0.180044
7282,1996-05-31,395056,"San Diego, CA",132244.0,-0.245155,-0.369755
...,...,...,...,...,...,...
7578,2021-01-31,395056,"San Diego, CA",498184.0,1.750774,1.645470
7579,2021-02-28,395056,"San Diego, CA",503827.0,1.132714,1.750774
7580,2021-03-31,395056,"San Diego, CA",512657.0,1.752586,1.132714
7581,2021-04-30,395056,"San Diego, CA",522174.0,1.856407,1.752586


In [79]:
DallasTX_condo_df = pd.DataFrame()
temp_df = metro_condo[metro_condo.region == majors_list[8]]
temp_df['change'] = temp_df['value'].pct_change() * 100
temp_df['lagged_change'] = temp_df['change'].shift()
DallasTX_condo_df = pd.concat([DallasTX_condo_df, temp_df])

DallasTX_condo_df

C:\ProgramData\Anaconda3\envs\deep\lib\site-packages\pandas\core\frame.py:3607: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._set_item(key, value)


,date,region_id,region,value,change,lagged_change
1830,1996-01-31,394514,"Dallas-Fort Worth, TX",88078.0,NaN,NaN
1831,1996-02-29,394514,"Dallas-Fort Worth, TX",87859.0,-0.248643,NaN
1832,1996-03-31,394514,"Dallas-Fort Worth, TX",88000.0,0.160484,-0.248643
1833,1996-04-30,394514,"Dallas-Fort Worth, TX",88448.0,0.509091,0.160484
1834,1996-05-31,394514,"Dallas-Fort Worth, TX",88943.0,0.559651,0.509091
...,...,...,...,...,...,...
2130,2021-01-31,394514,"Dallas-Fort Worth, TX",195811.0,0.536541,1.695393
2131,2021-02-28,394514,"Dallas-Fort Worth, TX",196151.0,0.173637,0.536541
2132,2021-03-31,394514,"Dallas-Fort Worth, TX",197987.0,0.936014,0.173637
2133,2021-04-30,394514,"Dallas-Fort Worth, TX",200709.0,1.374838,0.936014


In [81]:
SanJoseCA_condo_df = pd.DataFrame()
temp_df = metro_condo[metro_condo.region == majors_list[9]]
temp_df['change'] = temp_df['value'].pct_change() * 100
temp_df['lagged_change'] = temp_df['change'].shift()
SanJoseCA_condo_df = pd.concat([SanJoseCA_condo_df, temp_df])

SanJoseCA_condo_df

C:\ProgramData\Anaconda3\envs\deep\lib\site-packages\pandas\core\frame.py:3607: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._set_item(key, value)


,date,region_id,region,value,change,lagged_change
7888,1996-01-31,395059,"San Jose, CA",173950.0,NaN,NaN
7889,1996-02-29,395059,"San Jose, CA",173386.0,-0.324231,NaN
7890,1996-03-31,395059,"San Jose, CA",173065.0,-0.185136,-0.324231
7891,1996-04-30,395059,"San Jose, CA",172571.0,-0.285442,-0.185136
7892,1996-05-31,395059,"San Jose, CA",171579.0,-0.574836,-0.285442
...,...,...,...,...,...,...
8188,2021-01-31,395059,"San Jose, CA",842773.0,0.475929,1.755776
8189,2021-02-28,395059,"San Jose, CA",846505.0,0.442824,0.475929
8190,2021-03-31,395059,"San Jose, CA",817721.0,-3.400334,0.442824
8191,2021-04-30,395059,"San Jose, CA",825063.0,0.897861,-3.400334


In [82]:
AustinTX_condo_df = pd.DataFrame()
temp_df = metro_condo[metro_condo.region == majors_list[10]]
temp_df['change'] = temp_df['value'].pct_change() * 100
temp_df['lagged_change'] = temp_df['change'].shift()
AustinTX_condo_df = pd.concat([AustinTX_condo_df, temp_df])

AustinTX_condo_df

C:\ProgramData\Anaconda3\envs\deep\lib\site-packages\pandas\core\frame.py:3607: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._set_item(key, value)


,date,region_id,region,value,change,lagged_change
0,1996-01-31,394355,"Austin, TX",153184.0,NaN,NaN
1,1996-02-29,394355,"Austin, TX",152411.0,-0.504622,NaN
2,1996-03-31,394355,"Austin, TX",151035.0,-0.902822,-0.504622
3,1996-04-30,394355,"Austin, TX",149075.0,-1.297712,-0.902822
4,1996-05-31,394355,"Austin, TX",147382.0,-1.135670,-1.297712
...,...,...,...,...,...,...
300,2021-01-31,394355,"Austin, TX",358940.0,1.300761,5.812773
301,2021-02-28,394355,"Austin, TX",362015.0,0.856689,1.300761
302,2021-03-31,394355,"Austin, TX",375148.0,3.627750,0.856689
303,2021-04-30,394355,"Austin, TX",388194.0,3.477561,3.627750


In [83]:
JacksonvilleFL_condo_df = pd.DataFrame()
temp_df = metro_condo[metro_condo.region == majors_list[11]]
temp_df['change'] = temp_df['value'].pct_change() * 100
temp_df['lagged_change'] = temp_df['change'].shift()
JacksonvilleFL_condo_df = pd.concat([JacksonvilleFL_condo_df, temp_df])

JacksonvilleFL_condo_df

C:\ProgramData\Anaconda3\envs\deep\lib\site-packages\pandas\core\frame.py:3607: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._set_item(key, value)


,date,region_id,region,value,change,lagged_change
3618,1996-01-31,394714,"Jacksonville, FL",83053.0,NaN,NaN
3619,1996-02-29,394714,"Jacksonville, FL",83113.0,0.072243,NaN
3620,1996-03-31,394714,"Jacksonville, FL",83150.0,0.044518,0.072243
3621,1996-04-30,394714,"Jacksonville, FL",83245.0,0.114251,0.044518
3622,1996-05-31,394714,"Jacksonville, FL",83339.0,0.112920,0.114251
...,...,...,...,...,...,...
3918,2021-01-31,394714,"Jacksonville, FL",187921.0,1.497173,1.445948
3919,2021-02-28,394714,"Jacksonville, FL",190621.0,1.436774,1.497173
3920,2021-03-31,394714,"Jacksonville, FL",193945.0,1.743774,1.436774
3921,2021-04-30,394714,"Jacksonville, FL",196688.0,1.414318,1.743774


In [84]:
ColumbusOH_condo_df = pd.DataFrame()
temp_df = metro_condo[metro_condo.region == majors_list[12]]
temp_df['change'] = temp_df['value'].pct_change() * 100
temp_df['lagged_change'] = temp_df['change'].shift()
ColumbusOH_condo_df = pd.concat([ColumbusOH_condo_df, temp_df])

ColumbusOH_condo_df

C:\ProgramData\Anaconda3\envs\deep\lib\site-packages\pandas\core\frame.py:3607: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._set_item(key, value)


,date,region_id,region,value,change,lagged_change
1525,1996-01-31,394492,"Columbus, OH",91637.0,NaN,NaN
1526,1996-02-29,394492,"Columbus, OH",91790.0,0.166963,NaN
1527,1996-03-31,394492,"Columbus, OH",91962.0,0.187384,0.166963
1528,1996-04-30,394492,"Columbus, OH",92324.0,0.393641,0.187384
1529,1996-05-31,394492,"Columbus, OH",92728.0,0.437589,0.393641
...,...,...,...,...,...,...
1825,2021-01-31,394492,"Columbus, OH",173736.0,0.936534,1.322722
1826,2021-02-28,394492,"Columbus, OH",176032.0,1.321545,0.936534
1827,2021-03-31,394492,"Columbus, OH",178805.0,1.575282,1.321545
1828,2021-04-30,394492,"Columbus, OH",181008.0,1.232068,1.575282


In [85]:
CharlotteNC_condo_df = pd.DataFrame()
temp_df = metro_condo[metro_condo.region == majors_list[13]]
temp_df['change'] = temp_df['value'].pct_change() * 100
temp_df['lagged_change'] = temp_df['change'].shift()
CharlotteNC_condo_df = pd.concat([CharlotteNC_condo_df, temp_df])

CharlotteNC_condo_df

C:\ProgramData\Anaconda3\envs\deep\lib\site-packages\pandas\core\frame.py:3607: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._set_item(key, value)


,date,region_id,region,value,change,lagged_change
915,1996-01-31,394458,"Charlotte, NC",94489.0,NaN,NaN
916,1996-02-29,394458,"Charlotte, NC",94671.0,0.192615,NaN
917,1996-03-31,394458,"Charlotte, NC",94931.0,0.274635,0.192615
918,1996-04-30,394458,"Charlotte, NC",95474.0,0.571994,0.274635
919,1996-05-31,394458,"Charlotte, NC",96071.0,0.625301,0.571994
...,...,...,...,...,...,...
1215,2021-01-31,394458,"Charlotte, NC",207500.0,-1.993661,-2.012320
1216,2021-02-28,394458,"Charlotte, NC",209658.0,1.040000,-1.993661
1217,2021-03-31,394458,"Charlotte, NC",213754.0,1.953658,1.040000
1218,2021-04-30,394458,"Charlotte, NC",216704.0,1.380091,1.953658


In [86]:
IndianapolisIN_condo_df = pd.DataFrame()
temp_df = metro_condo[metro_condo.region == majors_list[14]]
temp_df['change'] = temp_df['value'].pct_change() * 100
temp_df['lagged_change'] = temp_df['change'].shift()
IndianapolisIN_condo_df = pd.concat([IndianapolisIN_condo_df, temp_df])

IndianapolisIN_condo_df

C:\ProgramData\Anaconda3\envs\deep\lib\site-packages\pandas\core\frame.py:3607: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._set_item(key, value)


,date,region_id,region,value,change,lagged_change
3355,1999-07-31,394705,"Indianapolis, IN",117318.0,NaN,NaN
3356,1999-08-31,394705,"Indianapolis, IN",116773.0,-0.464549,NaN
3357,1999-09-30,394705,"Indianapolis, IN",117023.0,0.214091,-0.464549
3358,1999-10-31,394705,"Indianapolis, IN",116946.0,-0.065799,0.214091
3359,1999-11-30,394705,"Indianapolis, IN",116466.0,-0.410446,-0.065799
...,...,...,...,...,...,...
3613,2021-01-31,394705,"Indianapolis, IN",167272.0,1.214413,-0.293211
3614,2021-02-28,394705,"Indianapolis, IN",169205.0,1.155603,1.214413
3615,2021-03-31,394705,"Indianapolis, IN",173639.0,2.620490,1.155603
3616,2021-04-30,394705,"Indianapolis, IN",175887.0,1.294640,2.620490


In [87]:
SanFrnaciscoCA_condo_df = pd.DataFrame()
temp_df = metro_condo[metro_condo.region == majors_list[15]]
temp_df['change'] = temp_df['value'].pct_change() * 100
temp_df['lagged_change'] = temp_df['change'].shift()
SanFrnaciscoCA_condo_df = pd.concat([SanFrnaciscoCA_condo_df, temp_df])

SanFrnaciscoCA_condo_df

C:\ProgramData\Anaconda3\envs\deep\lib\site-packages\pandas\core\frame.py:3607: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._set_item(key, value)


,date,region_id,region,value,change,lagged_change
7583,1996-01-31,395057,"San Francisco, CA",216933.0,NaN,NaN
7584,1996-02-29,395057,"San Francisco, CA",216107.0,-0.380763,NaN
7585,1996-03-31,395057,"San Francisco, CA",215674.0,-0.200364,-0.380763
7586,1996-04-30,395057,"San Francisco, CA",214821.0,-0.395504,-0.200364
7587,1996-05-31,395057,"San Francisco, CA",214361.0,-0.214132,-0.395504
...,...,...,...,...,...,...
7883,2021-01-31,395057,"San Francisco, CA",867371.0,0.774364,-0.123467
7884,2021-02-28,395057,"San Francisco, CA",869611.0,0.258252,0.774364
7885,2021-03-31,395057,"San Francisco, CA",860622.0,-1.033681,0.258252
7886,2021-04-30,395057,"San Francisco, CA",869454.0,1.026235,-1.033681


In [89]:
SeattleWA_condo_df = pd.DataFrame()
temp_df = metro_condo[metro_condo.region == majors_list[16]]
temp_df['change'] = temp_df['value'].pct_change() * 100
temp_df['lagged_change'] = temp_df['change'].shift()
SeattleWA_condo_df = pd.concat([SeattleWA_condo_df, temp_df])

SeattleWA_condo_df

C:\ProgramData\Anaconda3\envs\deep\lib\site-packages\pandas\core\frame.py:3607: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._set_item(key, value)


,date,region_id,region,value,change,lagged_change
8193,1996-01-31,395078,"Seattle, WA",130746.0,NaN,NaN
8194,1996-02-29,395078,"Seattle, WA",130672.0,-0.056598,NaN
8195,1996-03-31,395078,"Seattle, WA",130597.0,-0.057396,-0.056598
8196,1996-04-30,395078,"Seattle, WA",130476.0,-0.092651,-0.057396
8197,1996-05-31,395078,"Seattle, WA",130386.0,-0.068978,-0.092651
...,...,...,...,...,...,...
8493,2021-01-31,395078,"Seattle, WA",430120.0,0.756163,0.532226
8494,2021-02-28,395078,"Seattle, WA",430522.0,0.093462,0.756163
8495,2021-03-31,395078,"Seattle, WA",424942.0,-1.296101,0.093462
8496,2021-04-30,395078,"Seattle, WA",430612.0,1.334300,-1.296101


In [90]:
DenverCO_condo_df = pd.DataFrame()
temp_df = metro_condo[metro_condo.region == majors_list[17]]
temp_df['change'] = temp_df['value'].pct_change() * 100
temp_df['lagged_change'] = temp_df['change'].shift()
DenverCO_condo_df = pd.concat([DenverCO_condo_df, temp_df])

DenverCO_condo_df

C:\ProgramData\Anaconda3\envs\deep\lib\site-packages\pandas\core\frame.py:3607: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._set_item(key, value)


,date,region_id,region,value,change,lagged_change
2135,1996-01-31,394530,"Denver, CO",91514.0,NaN,NaN
2136,1996-02-29,394530,"Denver, CO",91872.0,0.391197,NaN
2137,1996-03-31,394530,"Denver, CO",92142.0,0.293887,0.391197
2138,1996-04-30,394530,"Denver, CO",92695.0,0.600161,0.293887
2139,1996-05-31,394530,"Denver, CO",93142.0,0.482227,0.600161
...,...,...,...,...,...,...
2435,2021-01-31,394530,"Denver, CO",298621.0,1.219909,1.068852
2436,2021-02-28,394530,"Denver, CO",302096.0,1.163682,1.219909
2437,2021-03-31,394530,"Denver, CO",305541.0,1.140366,1.163682
2438,2021-04-30,394530,"Denver, CO",309673.0,1.352355,1.140366


In [92]:
WashingtonDC_condo_df = pd.DataFrame()
temp_df = metro_condo[metro_condo.region == majors_list[18]]
temp_df['change'] = temp_df['value'].pct_change() * 100
temp_df['lagged_change'] = temp_df['change'].shift()
WashingtonDC_condo_df = pd.concat([WashingtonDC_condo_df, temp_df])

WashingtonDC_condo_df

C:\ProgramData\Anaconda3\envs\deep\lib\site-packages\pandas\core\frame.py:3607: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._set_item(key, value)


,date,region_id,region,value,change,lagged_change
8498,1996-01-31,395209,"Washington, DC",124239.0,NaN,NaN
8499,1996-02-29,395209,"Washington, DC",124004.0,-0.189152,NaN
8500,1996-03-31,395209,"Washington, DC",123859.0,-0.116932,-0.189152
8501,1996-04-30,395209,"Washington, DC",123538.0,-0.259166,-0.116932
8502,1996-05-31,395209,"Washington, DC",123307.0,-0.186987,-0.259166
...,...,...,...,...,...,...
8798,2021-01-31,395209,"Washington, DC",323745.0,-0.168983,2.239352
8799,2021-02-28,395209,"Washington, DC",327560.0,1.178397,-0.168983
8800,2021-03-31,395209,"Washington, DC",327932.0,0.113567,1.178397
8801,2021-04-30,395209,"Washington, DC",330330.0,0.731249,0.113567


In [94]:
BostonMA_condo_df = pd.DataFrame()
temp_df = metro_condo[metro_condo.region == majors_list[19]]
temp_df['change'] = temp_df['value'].pct_change() * 100
temp_df['lagged_change'] = temp_df['change'].shift()
BostonMA_condo_df = pd.concat([BostonMA_condo_df, temp_df])

BostonMA_condo_df

C:\ProgramData\Anaconda3\envs\deep\lib\site-packages\pandas\core\frame.py:3607: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._set_item(key, value)


,date,region_id,region,value,change,lagged_change
610,1996-01-31,394404,"Boston, MA",129319.0,NaN,NaN
611,1996-02-29,394404,"Boston, MA",129318.0,-0.000773,NaN
612,1996-03-31,394404,"Boston, MA",129411.0,0.071916,-0.000773
613,1996-04-30,394404,"Boston, MA",129634.0,0.172319,0.071916
614,1996-05-31,394404,"Boston, MA",129939.0,0.235278,0.172319
...,...,...,...,...,...,...
910,2021-01-31,394404,"Boston, MA",456505.0,1.073381,0.539363
911,2021-02-28,394404,"Boston, MA",460478.0,0.870308,1.073381
912,2021-03-31,394404,"Boston, MA",464000.0,0.764857,0.870308
913,2021-04-30,394404,"Boston, MA",468258.0,0.917672,0.764857


In [96]:
ElPaso_condo_df = pd.DataFrame()
temp_df = metro_condo[metro_condo.region == majors_list[20]]
temp_df['change'] = temp_df['value'].pct_change() * 100
temp_df['lagged_change'] = temp_df['change'].shift()
ElPaso_condo_df = pd.concat([ElPaso_condo_df, temp_df])

ElPaso_condo_df

C:\ProgramData\Anaconda3\envs\deep\lib\site-packages\pandas\core\frame.py:3607: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._set_item(key, value)


,date,region_id,region,value,change,lagged_change
2745,1996-01-31,394561,"El Paso, TX",78992.0,NaN,NaN
2746,1996-02-29,394561,"El Paso, TX",78761.0,-0.292435,NaN
2747,1996-03-31,394561,"El Paso, TX",78616.0,-0.184101,-0.292435
2748,1996-04-30,394561,"El Paso, TX",78424.0,-0.244225,-0.184101
2749,1996-05-31,394561,"El Paso, TX",78454.0,0.038254,-0.244225
...,...,...,...,...,...,...
3045,2021-01-31,394561,"El Paso, TX",114784.0,1.447687,1.862672
3046,2021-02-28,394561,"El Paso, TX",114186.0,-0.520979,1.447687
3047,2021-03-31,394561,"El Paso, TX",116027.0,1.612282,-0.520979
3048,2021-04-30,394561,"El Paso, TX",117555.0,1.316935,1.612282


In [98]:
NashvilleTN_condo_df = pd.DataFrame()
temp_df = metro_condo[metro_condo.region == majors_list[21]]
temp_df['change'] = temp_df['value'].pct_change() * 100
temp_df['lagged_change'] = temp_df['change'].shift()
NashvilleTN_condo_df = pd.concat([NashvilleTN_condo_df, temp_df])

NashvilleTN_condo_df

C:\ProgramData\Anaconda3\envs\deep\lib\site-packages\pandas\core\frame.py:3607: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._set_item(key, value)


,date,region_id,region,value,change,lagged_change
5143,1996-01-31,394902,"Nashville, TN",106353.0,NaN,NaN
5144,1996-02-29,394902,"Nashville, TN",106765.0,0.387389,NaN
5145,1996-03-31,394902,"Nashville, TN",107154.0,0.364352,0.387389
5146,1996-04-30,394902,"Nashville, TN",107990.0,0.780186,0.364352
5147,1996-05-31,394902,"Nashville, TN",108766.0,0.718585,0.780186
...,...,...,...,...,...,...
5443,2021-01-31,394902,"Nashville, TN",267388.0,1.666122,0.496357
5444,2021-02-28,394902,"Nashville, TN",275783.0,3.139632,1.666122
5445,2021-03-31,394902,"Nashville, TN",283554.0,2.817795,3.139632
5446,2021-04-30,394902,"Nashville, TN",286520.0,1.046009,2.817795


In [100]:
DetroitMI_condo_df = pd.DataFrame()
temp_df = metro_condo[metro_condo.region == majors_list[22]]
temp_df['change'] = temp_df['value'].pct_change() * 100
temp_df['lagged_change'] = temp_df['change'].shift()
DetroitMI_condo_df = pd.concat([DetroitMI_condo_df, temp_df])

DetroitMI_condo_df

C:\ProgramData\Anaconda3\envs\deep\lib\site-packages\pandas\core\frame.py:3607: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._set_item(key, value)


,date,region_id,region,value,change,lagged_change
2440,1996-01-31,394532,"Detroit, MI",123341.0,NaN,NaN
2441,1996-02-29,394532,"Detroit, MI",123718.0,0.305657,NaN
2442,1996-03-31,394532,"Detroit, MI",124052.0,0.269969,0.305657
2443,1996-04-30,394532,"Detroit, MI",124806.0,0.607810,0.269969
2444,1996-05-31,394532,"Detroit, MI",125558.0,0.602535,0.607810
...,...,...,...,...,...,...
2740,2021-01-31,394532,"Detroit, MI",208288.0,2.410195,1.624911
2741,2021-02-28,394532,"Detroit, MI",209976.0,0.810416,2.410195
2742,2021-03-31,394532,"Detroit, MI",210312.0,0.160018,0.810416
2743,2021-04-30,394532,"Detroit, MI",212513.0,1.046540,0.160018


In [102]:
LasVegasNV_condo_df = pd.DataFrame()
temp_df = metro_condo[metro_condo.region == majors_list[23]]
temp_df['change'] = temp_df['value'].pct_change() * 100
temp_df['lagged_change'] = temp_df['change'].shift()
LasVegasNV_condo_df = pd.concat([LasVegasNV_condo_df, temp_df])

LasVegasNV_condo_df

C:\ProgramData\Anaconda3\envs\deep\lib\site-packages\pandas\core\frame.py:3607: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._set_item(key, value)


,date,region_id,region,value,change,lagged_change
3923,1996-01-31,394775,"Las Vegas, NV",96609.0,NaN,NaN
3924,1996-02-29,394775,"Las Vegas, NV",96526.0,-0.085913,NaN
3925,1996-03-31,394775,"Las Vegas, NV",96510.0,-0.016576,-0.085913
3926,1996-04-30,394775,"Las Vegas, NV",96401.0,-0.112942,-0.016576
3927,1996-05-31,394775,"Las Vegas, NV",96398.0,-0.003112,-0.112942
...,...,...,...,...,...,...
4223,2021-01-31,394775,"Las Vegas, NV",171212.0,1.096513,0.866587
4224,2021-02-28,394775,"Las Vegas, NV",173260.0,1.196178,1.096513
4225,2021-03-31,394775,"Las Vegas, NV",173390.0,0.075032,1.196178
4226,2021-04-30,394775,"Las Vegas, NV",175643.0,1.299383,0.075032


In [104]:
OklahomaCityOK_condo_df = pd.DataFrame()
temp_df = metro_condo[metro_condo.region == majors_list[24]]
temp_df['change'] = temp_df['value'].pct_change() * 100
temp_df['lagged_change'] = temp_df['change'].shift()
OklahomaCityOK_condo_df = pd.concat([OklahomaCityOK_condo_df, temp_df])

OklahomaCityOK_condo_df

C:\ProgramData\Anaconda3\envs\deep\lib\site-packages\pandas\core\frame.py:3607: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._set_item(key, value)


,date,region_id,region,value,change,lagged_change
5753,1996-01-31,394935,"Oklahoma City, OK",44554.0,NaN,NaN
5754,1996-02-29,394935,"Oklahoma City, OK",44594.0,0.089779,NaN
5755,1996-03-31,394935,"Oklahoma City, OK",44806.0,0.475400,0.089779
5756,1996-04-30,394935,"Oklahoma City, OK",45095.0,0.645003,0.475400
5757,1996-05-31,394935,"Oklahoma City, OK",45509.0,0.918062,0.645003
...,...,...,...,...,...,...
6053,2021-01-31,394935,"Oklahoma City, OK",92594.0,1.382882,-0.629964
6054,2021-02-28,394935,"Oklahoma City, OK",92378.0,-0.233276,1.382882
6055,2021-03-31,394935,"Oklahoma City, OK",97599.0,5.651779,-0.233276
6056,2021-04-30,394935,"Oklahoma City, OK",99035.0,1.471327,5.651779


In [105]:
PortlandOR_condo_df = pd.DataFrame()
temp_df = metro_condo[metro_condo.region == majors_list[25]]
temp_df['change'] = temp_df['value'].pct_change() * 100
temp_df['lagged_change'] = temp_df['change'].shift()
PortlandOR_condo_df = pd.concat([PortlandOR_condo_df, temp_df])

PortlandOR_condo_df

C:\ProgramData\Anaconda3\envs\deep\lib\site-packages\pandas\core\frame.py:3607: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._set_item(key, value)


,date,region_id,region,value,change,lagged_change
6668,1996-01-31,394998,"Portland, OR",119046.0,NaN,NaN
6669,1996-02-29,394998,"Portland, OR",119509.0,0.388925,NaN
6670,1996-03-31,394998,"Portland, OR",119911.0,0.336376,0.388925
6671,1996-04-30,394998,"Portland, OR",120844.0,0.778077,0.336376
6672,1996-05-31,394998,"Portland, OR",121779.0,0.773725,0.778077
...,...,...,...,...,...,...
6968,2021-01-31,394998,"Portland, OR",322672.0,1.343300,2.353788
6969,2021-02-28,394998,"Portland, OR",325250.0,0.798954,1.343300
6970,2021-03-31,394998,"Portland, OR",325320.0,0.021522,0.798954
6971,2021-04-30,394998,"Portland, OR",328988.0,1.127505,0.021522


In [107]:
MemphisTN_condo_df = pd.DataFrame()
temp_df = metro_condo[metro_condo.region == majors_list[26]]
temp_df['change'] = temp_df['value'].pct_change() * 100
temp_df['lagged_change'] = temp_df['change'].shift()
MemphisTN_condo_df = pd.concat([MemphisTN_condo_df, temp_df])

MemphisTN_condo_df

C:\ProgramData\Anaconda3\envs\deep\lib\site-packages\pandas\core\frame.py:3607: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._set_item(key, value)


,date,region_id,region,value,change,lagged_change
4533,1996-01-31,394849,"Memphis, TN",148402.0,NaN,NaN
4534,1996-02-29,394849,"Memphis, TN",148736.0,0.225064,NaN
4535,1996-03-31,394849,"Memphis, TN",149174.0,0.294481,0.225064
4536,1996-04-30,394849,"Memphis, TN",150041.0,0.581200,0.294481
4537,1996-05-31,394849,"Memphis, TN",150952.0,0.607167,0.581200
...,...,...,...,...,...,...
4833,2021-01-31,394849,"Memphis, TN",194962.0,1.772757,1.082770
4834,2021-02-28,394849,"Memphis, TN",200994.0,3.093936,1.772757
4835,2021-03-31,394849,"Memphis, TN",202468.0,0.733355,3.093936
4836,2021-04-30,394849,"Memphis, TN",204764.0,1.134006,0.733355


In [108]:
LouisvilleKY_condo_df = pd.DataFrame()
temp_df = metro_condo[metro_condo.region == majors_list[27]]
temp_df['change'] = temp_df['value'].pct_change() * 100
temp_df['lagged_change'] = temp_df['change'].shift()
LouisvilleKY_condo_df = pd.concat([LouisvilleKY_condo_df, temp_df])

LouisvilleKY_condo_df

C:\ProgramData\Anaconda3\envs\deep\lib\site-packages\pandas\core\frame.py:3607: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._set_item(key, value)


,date,region_id,region,value,change,lagged_change
4228,1996-01-31,394807,"Louisville-Jefferson County, KY",96443.0,NaN,NaN
4229,1996-02-29,394807,"Louisville-Jefferson County, KY",96579.0,0.141016,NaN
4230,1996-03-31,394807,"Louisville-Jefferson County, KY",96671.0,0.095259,0.141016
4231,1996-04-30,394807,"Louisville-Jefferson County, KY",96836.0,0.170682,0.095259
4232,1996-05-31,394807,"Louisville-Jefferson County, KY",96953.0,0.120823,0.170682
...,...,...,...,...,...,...
4528,2021-01-31,394807,"Louisville-Jefferson County, KY",162861.0,0.175303,0.641327
4529,2021-02-28,394807,"Louisville-Jefferson County, KY",165549.0,1.650487,0.175303
4530,2021-03-31,394807,"Louisville-Jefferson County, KY",165840.0,0.175779,1.650487
4531,2021-04-30,394807,"Louisville-Jefferson County, KY",167657.0,1.095634,0.175779


In [109]:
MilwaukeeWI_condo_df = pd.DataFrame()
temp_df = metro_condo[metro_condo.region == majors_list[28]]
temp_df['change'] = temp_df['value'].pct_change() * 100
temp_df['lagged_change'] = temp_df['change'].shift()
MilwaukeeWI_condo_df = pd.concat([MilwaukeeWI_condo_df, temp_df])

MilwaukeeWI_condo_df

C:\ProgramData\Anaconda3\envs\deep\lib\site-packages\pandas\core\frame.py:3607: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._set_item(key, value)


,date,region_id,region,value,change,lagged_change
4838,1996-01-31,394862,"Milwaukee, WI",109681.0,NaN,NaN
4839,1996-02-29,394862,"Milwaukee, WI",110164.0,0.440368,NaN
4840,1996-03-31,394862,"Milwaukee, WI",110314.0,0.136161,0.440368
4841,1996-04-30,394862,"Milwaukee, WI",110819.0,0.457784,0.136161
4842,1996-05-31,394862,"Milwaukee, WI",110972.0,0.138063,0.457784
...,...,...,...,...,...,...
5138,2021-01-31,394862,"Milwaukee, WI",195230.0,1.800529,2.198763
5139,2021-02-28,394862,"Milwaukee, WI",196772.0,0.789838,1.800529
5140,2021-03-31,394862,"Milwaukee, WI",198921.0,1.092127,0.789838
5141,2021-04-30,394862,"Milwaukee, WI",201065.0,1.077815,1.092127


In [111]:
BaltimoreMD_condo_df = pd.DataFrame()
temp_df = metro_condo[metro_condo.region == majors_list[29]]
temp_df['change'] = temp_df['value'].pct_change() * 100
temp_df['lagged_change'] = temp_df['change'].shift()
BaltimoreMD_condo_df = pd.concat([BaltimoreMD_condo_df, temp_df])

BaltimoreMD_condo_df

C:\ProgramData\Anaconda3\envs\deep\lib\site-packages\pandas\core\frame.py:3607: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._set_item(key, value)


,date,region_id,region,value,change,lagged_change
305,1996-01-31,394358,"Baltimore, MD",115806.0,NaN,NaN
306,1996-02-29,394358,"Baltimore, MD",115774.0,-0.027632,NaN
307,1996-03-31,394358,"Baltimore, MD",115771.0,-0.002591,-0.027632
308,1996-04-30,394358,"Baltimore, MD",115821.0,0.043189,-0.002591
309,1996-05-31,394358,"Baltimore, MD",115884.0,0.054394,0.043189
...,...,...,...,...,...,...
605,2021-01-31,394358,"Baltimore, MD",220547.0,0.111211,2.054506
606,2021-02-28,394358,"Baltimore, MD",223693.0,1.426453,0.111211
607,2021-03-31,394358,"Baltimore, MD",225077.0,0.618705,1.426453
608,2021-04-30,394358,"Baltimore, MD",227799.0,1.209364,0.618705


In [113]:
BaltimoreMD_condo_df.to_csv('./Resources/BalMD_con.csv', index=False)

### Create Single Bedroom DataFrame

In [194]:
metro_1br = metro_homes[metro_homes.indicator_id == 'Z1BR']
metro_1br = metro_1br.sort_values(by='date')
metro_1br = metro_1br.drop(columns='indicator_id')
metro_1br

,region_id,date,value,region
50405,394935,1996-01-31,23003.0,"Oklahoma City, OK"
21380,395078,1996-01-31,110085.0,"Seattle, WA"
23210,395056,1996-01-31,91088.0,"San Diego, CA"
39425,395059,1996-01-31,137890.0,"San Jose, CA"
54065,394561,1996-01-31,70746.0,"El Paso, TX"
...,...,...,...,...
18378,394976,2021-05-31,200496.0,"Phoenix, AZ"
43743,394862,2021-05-31,149830.0,"Milwaukee, WI"
36678,394705,2021-05-31,117388.0,"Indianapolis, IN"
1908,753899,2021-05-31,488200.0,"Los Angeles-Long Beach-Anaheim, CA"


In [195]:
metro_1br = metro_1br.reindex(columns=['date', 'region_id', 'region', 'value'])
metro_1br

,date,region_id,region,value
50405,1996-01-31,394935,"Oklahoma City, OK",23003.0
21380,1996-01-31,395078,"Seattle, WA",110085.0
23210,1996-01-31,395056,"San Diego, CA",91088.0
39425,1996-01-31,395059,"San Jose, CA",137890.0
54065,1996-01-31,394561,"El Paso, TX",70746.0
...,...,...,...,...
18378,2021-05-31,394976,"Phoenix, AZ",200496.0
43743,2021-05-31,394862,"Milwaukee, WI",149830.0
36678,2021-05-31,394705,"Indianapolis, IN",117388.0
1908,2021-05-31,753899,"Los Angeles-Long Beach-Anaheim, CA",488200.0


In [196]:
metro_1br.to_csv('./Resources/metro_1br.csv', index=False)

### Create Two Bedroom DataFrame

In [197]:
metro_2br = metro_homes[metro_homes.indicator_id == 'Z2BR']
metro_2br = metro_2br.sort_values(by='date')
metro_2br = metro_2br.drop(columns='indicator_id')
metro_2br

,region_id,date,value,region
26290,394530,1996-01-31,106036.0,"Denver, CO"
6160,394514,1996-01-31,71763.0,"Dallas-Fort Worth, TX"
49825,394935,1996-01-31,34698.0,"Oklahoma City, OK"
46165,394714,1996-01-31,69888.0,"Jacksonville, FL"
42505,394902,1996-01-31,82673.0,"Nashville, TN"
...,...,...,...,...
18372,394976,2021-05-31,299023.0,"Phoenix, AZ"
43737,394862,2021-05-31,181716.0,"Milwaukee, WI"
36672,394705,2021-05-31,142371.0,"Indianapolis, IN"
1902,753899,2021-05-31,626648.0,"Los Angeles-Long Beach-Anaheim, CA"


In [198]:
metro_2br = metro_2br.reindex(columns=['date', 'region_id', 'region', 'value'])
metro_2br

,date,region_id,region,value
26290,1996-01-31,394530,"Denver, CO",106036.0
6160,1996-01-31,394514,"Dallas-Fort Worth, TX",71763.0
49825,1996-01-31,394935,"Oklahoma City, OK",34698.0
46165,1996-01-31,394714,"Jacksonville, FL",69888.0
42505,1996-01-31,394902,"Nashville, TN",82673.0
...,...,...,...,...
18372,2021-05-31,394976,"Phoenix, AZ",299023.0
43737,2021-05-31,394862,"Milwaukee, WI",181716.0
36672,2021-05-31,394705,"Indianapolis, IN",142371.0
1902,2021-05-31,753899,"Los Angeles-Long Beach-Anaheim, CA",626648.0


In [199]:
metro_2br.to_csv('./Resources/metro_2br.csv', index=False)

### Create Three Bedroom DataFrame

In [200]:
metro_3br = metro_homes[metro_homes.indicator_id == 'Z3BR']
metro_3br = metro_3br.sort_values(by='date')
metro_3br = metro_3br.drop(columns='indicator_id')
metro_3br

,region_id,date,value,region
21090,395078,1996-01-31,158380.0,"Seattle, WA"
13770,394404,1996-01-31,161813.0,"Boston, MA"
10110,394692,1996-01-31,90655.0,"Houston, TX"
39135,395059,1996-01-31,233179.0,"San Jose, CA"
44625,394862,1996-01-31,104849.0,"Milwaukee, WI"
...,...,...,...,...
18375,394976,2021-05-31,344737.0,"Phoenix, AZ"
43740,394862,2021-05-31,237699.0,"Milwaukee, WI"
36675,394705,2021-05-31,202488.0,"Indianapolis, IN"
1905,753899,2021-05-31,760384.0,"Los Angeles-Long Beach-Anaheim, CA"


In [201]:
metro_3br = metro_3br.reindex(columns=['date', 'region_id', 'region', 'value'])
metro_3br

,date,region_id,region,value
21090,1996-01-31,395078,"Seattle, WA",158380.0
13770,1996-01-31,394404,"Boston, MA",161813.0
10110,1996-01-31,394692,"Houston, TX",90655.0
39135,1996-01-31,395059,"San Jose, CA",233179.0
44625,1996-01-31,394862,"Milwaukee, WI",104849.0
...,...,...,...,...
18375,2021-05-31,394976,"Phoenix, AZ",344737.0
43740,2021-05-31,394862,"Milwaukee, WI",237699.0
36675,2021-05-31,394705,"Indianapolis, IN",202488.0
1905,2021-05-31,753899,"Los Angeles-Long Beach-Anaheim, CA",760384.0


In [202]:
metro_3br.to_csv('./Resources/metro_3br.csv', index=False)

### Create Four Bedroom DataFrame

In [203]:
metro_4br = metro_homes[metro_homes.indicator_id == 'Z4BR']
metro_4br = metro_4br.sort_values(by='date')
metro_4br = metro_4br.drop(columns='indicator_id')
metro_4br

,region_id,date,value,region
21670,395078,1996-01-31,202353.0,"Seattle, WA"
14350,394404,1996-01-31,230241.0,"Boston, MA"
10690,394692,1996-01-31,148568.0,"Houston, TX"
39715,395059,1996-01-31,278763.0,"San Jose, CA"
45205,394862,1996-01-31,136670.0,"Milwaukee, WI"
...,...,...,...,...
38256,395059,2021-05-31,1587759.0,"San Jose, CA"
27531,394998,2021-05-31,585879.0,"Portland, OR"
29361,394458,2021-05-31,384692.0,"Charlotte, NC"
20211,395078,2021-05-31,769318.0,"Seattle, WA"


In [204]:
metro_4br = metro_4br.reindex(columns=['date', 'region_id', 'region', 'value'])
metro_4br

,date,region_id,region,value
21670,1996-01-31,395078,"Seattle, WA",202353.0
14350,1996-01-31,394404,"Boston, MA",230241.0
10690,1996-01-31,394692,"Houston, TX",148568.0
39715,1996-01-31,395059,"San Jose, CA",278763.0
45205,1996-01-31,394862,"Milwaukee, WI",136670.0
...,...,...,...,...
38256,2021-05-31,395059,"San Jose, CA",1587759.0
27531,2021-05-31,394998,"Portland, OR",585879.0
29361,2021-05-31,394458,"Charlotte, NC",384692.0
20211,2021-05-31,395078,"Seattle, WA",769318.0


In [205]:
metro_4br.to_csv('./Resources/metro_4br.csv', index=False)

### Create Five+ Bedroom DataFrame

In [206]:
metro_5br = metro_homes[metro_homes.indicator_id == 'Z5BR']
metro_5br = metro_5br.sort_values(by='date')
metro_5br = metro_5br.drop(columns='indicator_id')
metro_5br

,region_id,date,value,region
20510,395078,1996-01-31,231853.0,"Seattle, WA"
13190,394404,1996-01-31,310862.0,"Boston, MA"
9530,394692,1996-01-31,232670.0,"Houston, TX"
38555,395059,1996-01-31,453298.0,"San Jose, CA"
44045,394862,1996-01-31,159228.0,"Milwaukee, WI"
...,...,...,...,...
18369,394976,2021-05-31,591042.0,"Phoenix, AZ"
43734,394862,2021-05-31,347785.0,"Milwaukee, WI"
36669,394705,2021-05-31,452767.0,"Indianapolis, IN"
1899,753899,2021-05-31,1364625.0,"Los Angeles-Long Beach-Anaheim, CA"


In [207]:
metro_5br = metro_5br.reindex(columns=['date', 'region_id', 'region', 'value'])
metro_5br

,date,region_id,region,value
20510,1996-01-31,395078,"Seattle, WA",231853.0
13190,1996-01-31,394404,"Boston, MA",310862.0
9530,1996-01-31,394692,"Houston, TX",232670.0
38555,1996-01-31,395059,"San Jose, CA",453298.0
44045,1996-01-31,394862,"Milwaukee, WI",159228.0
...,...,...,...,...
18369,2021-05-31,394976,"Phoenix, AZ",591042.0
43734,2021-05-31,394862,"Milwaukee, WI",347785.0
36669,2021-05-31,394705,"Indianapolis, IN",452767.0
1899,2021-05-31,753899,"Los Angeles-Long Beach-Anaheim, CA",1364625.0


In [208]:
metro_5br.to_csv('./Resources/metro_5br.csv', index=False)

In [209]:
metro_condo

,date,region_id,region,value
29370,1996-01-31,394458,"Charlotte, NC",94489.0
33030,1996-01-31,394775,"Las Vegas, NV",96609.0
52905,1996-01-31,394561,"El Paso, TX",78992.0
41925,1996-01-31,394902,"Nashville, TN",106353.0
40095,1996-01-31,394355,"Austin, TX",153184.0
...,...,...,...,...
18366,2021-05-31,394976,"Phoenix, AZ",245235.0
43731,2021-05-31,394862,"Milwaukee, WI",203682.0
36666,2021-05-31,394705,"Indianapolis, IN",178355.0
1896,2021-05-31,753899,"Los Angeles-Long Beach-Anaheim, CA",604688.0
